* **gradient boosting**은 많은 Kaggle competition에서 우위를 차지하고, 다양한 데이터셋에서 state-of-the-art 결과를 차지함

## Introduction

* random forest를 "앙상블 메소드"라고 함
* **앙상블 메소드(ensembel method)**는 여러 모델의 예측을 결합함(e.g., random forest의 경우에, 여러 트리)

<br>

## Gradient Boosting

* **gradient boosting**은 주기를 거쳐, 반복적으로 앙상블에 모델을 추가하는 메소드
* gradient boosting은 단일 모델로 앙상블을 추가하는 것부터 시작함. 단일 모델이라 예측이 단순함(예측이 많이 부정확해도, 이 다음 앙상블에 추가되는 모델이 error를 조절함)
* 주기 시작:
    * 먼저, 데이터셋에서 각 관찰(observation)별로 예측을 만들기 위해 현재의 앙상블을 사용. 전체 예측을 위해, 앙상블의 모든 모델로부터의 예측을 추가함
    * 이 예측들은 loss function(e.g. [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error))을 계산하는데 사용
    * 그 다음, 계산한 loss function을 앙상블에 추가될 새로운 모델을 fitting하는 데 사용. 앙상블에 이 새로운 모델을 추가하면 loss를 줄일 수 있도록 모델 파라미터를 결정(*Side note*: "gradient boosting"의 "gradient"는 새로운 모델의 파라미터를 결정하기 위한 loss function에 [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)를 사용한다는 것을 의미함)
    * 마지막으로, 새로운 모델을 앙상블에 추가
    * 위 과정을 반복
    
    <img src="../images/3/EnsembleCycle.png" width="100%">
    
<br>

## Example

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

* `XGBoost`: 성능과 속도에 집중하는 몇 개의 추가적인 기능을 사용하여 gradient boosting을 구현한 **extreme gradient boosting**을 나타냄.(scikit-learn에 또 다른 버전의 gradient boosting이 있지만, XGBoost가 몇몇 기술적 이점을 가지고 있음)

In [4]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

[15:42:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [5]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 267493.7275497054


<br>

## Parameter Tuning

* XGBoost는 정확도와 학습 속도에 드라마틱한 영향을 줄 수 있는 몇몇 파라미터가 있음
* **n_estimators**: 모델링 주기를 얼마나 많이 지날 수 있는지를 명시. 즉, 앙상블에 추가할 수 있는 모델의 수와 같음
    * 너무 낮은 값은 *underfitting*을 일으킬 수 있음(학습 데이터와 테스트 데이터 모두에 부정확한 결과)
    * 너무 높은 값은 *overfitting*을 일으킬 수 있음(학습 데이터에는 정확한 결과를 내지만, 테스트 데이터에는 부정확한 결과)
* 일반적으로 100~1000 사이의 값을 사용하지만, learning rate에 따라 크게 달라짐.

In [7]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

[15:42:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

* **early_stopping_rounds**: 자동으로 n_estimators의 이상적인 값을 찾음. validation score가 더 이상 증가하지 않을 때, n_esrimators 값이 설정되어 있어도 주기의 반복을 중지함. n_estimators의 값을 높게 설정한 다음, early_stopping_rounds를 사용해 반복을 중지할 최적의 값을 찾으면 좋음
   
   <br>
   가끔 한 번만에 validation score가 증가하지 않을 수 있기 때문에, 반복을 중지하기 전에 (그래도) 얼마나 반복할지 횟수를 정해야함. 이 코스에선 `early_stopping_rounds=5`로 설정
   
   <br>
   early_stopping_rounds를 사용할 때, validation score를 계산하기 위한 일부 데이터를 정해야함 - `eval_set` 사용

In [9]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False) ## verbose: 학습 중 출력될 문구 설정(0: silent, 1: progress bar, 2:one line per epoch)

[15:42:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

* 나중에 모델이 모든 데이터에 fitting되길 원하면, early stopping 했을 때 찾을 수 있는 최적의 값을 `n_estimators`로 설정하고 eval_set을 빼면 됨(early stopping도 사용 X)
   
* **learning_rate**: 각 모델의 예측을 더해서 전체 예측을 얻는 대신에, 각 모델에 작은 수(learning rate라고도 함)를 곱하고 더해서 전체 예측을 얻을 수 있음<br>
이것은 앙상블에 더해진 각 트리가 도움이 덜 된다는 것을 의미. 그래서 overfitting 없이 n_estimators 값을 더 높게 설정할 수 있음.(early stopping을 사용하면, 적절한 트리 수가 자동으로 결정됨)
* 일반적으로 작은 learning rate와 큰 estimators는 XGBoost model의 학습을 더 길게 만들지만, 더 높은 정확도를 내게 함

In [10]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

[15:42:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

* **n_jobs**: 실행시간이 고려되는 큰 데이터셋에서, 모델을 더 빠르게 빌드할 수 있게 하는 **parallelism**을 사용할 수 있음
* **n_jobs** 파라미터를 머신의 core 수와 동일하게 설정하는 것이 일반적(작은 데이터셋에선 별로 도움이 되지 않음)

In [12]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

[15:54:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)